In [ ]:
import urllib
from pyspark.sql.types import *
from IPython.display import Javascript
import json

bankText = sc.parallelize(urllib.urlopen('https://s3.amazonaws.com/apache-zeppelin/tutorial/bank/bank.csv'))
bank = bankText.map(lambda s: s.split(";")).filter(lambda s: s[0] != "\"age\"").map(lambda s:
        (int(s[0]), 
            s[1].replace("\"", ""),
            s[2].replace("\"", ""),
            s[3].replace("\"", ""),
            int(s[5].replace("\"", ""))
         )
)

# age: Integer, job: String, marital: String, education: String, balance: Integer
Bank = StructType([
        StructField('age', IntegerType(), True), 
        StructField('job', StringType(), True),
        StructField('marital', StringType(), True),
        StructField('education', StringType(), True),
        StructField('balance', IntegerType(), True)
    ])
bankDF = sqlContext.createDataFrame(bank, Bank)
bankDF.registerTempTable("bank")


values = sqlContext.sql("""
    select age, count(1) value
    from bank 
    where age > 10
    group by age 
    order by age
""")

values = values.map(lambda x: x.asDict()).collect()

Javascript("window.vizObj = %s" % json.dumps(values))


In [ ]:
%%javascript

element.append("<div id='chartContainer'></div>");

setTimeout(function (){ // workaround because theres no way to run this after append is complete
    var svg = dimple.newSvg("#chartContainer", 590, 400);
    var myChart = new dimple.chart(svg, vizObj);
    myChart.setBounds(60, 30, 510, 305)
    myChart.addCategoryAxis("x", "age");
    myChart.addMeasureAxis("y", "value");
    myChart.addSeries(null, dimple.plot.bar);
    myChart.draw();
}, 1);